In [7]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [8]:
#matrix: maxtrix[n,m] = list of size nxm
#vector: vector[m,1] = list of size m
#sc: SparkContext
def multiplyMatrixByVector(matrix,vector,sc):
    
    matrixRDD = sc.parallelize(matrix,8) #matriz reducida la paralelizo en RDDs
    
    matrixPerRowRDD = matrixRDD.map(lambda x: (x[0],(x[1],x[2]))) #(fila,(columna,valor))
    
    #mutiplico cada elemento de una fila por el valor del vector correspondiente
    partialResultRDD = matrixPerRowRDD.map(lambda x: (x[0], vector[x[1][0]-1] * x[1][1])) 
    
    #sumo las que tienen la misma key (fila)
    resultPerRow = partialResultRDD.reduceByKey(lambda x,y: x+y)
    
    return resultPerRow.map(lambda x: x[1]) #devuelvo un vector en forma de lista

Pruebo mi función definiendo una matriz y un vector

In [10]:
matrix = [
(1,2,4),
(1,5,3),
(2,1,3),
(3,2,2),
(4,4,-1),
(5,1,1),
(5,5,2)]

vector = [1, 2, 3, 4, 5]

print(multiplyMatrixByVector(matrix,vector,sc).take(20))



[23, 3, 4, -4, 11]
